# ArchiCrop tutorial - Tillering

This notebook is a tutorial for understanding the tillering model in ArchiCrop model.

## 0. Imports

In [1]:
import numpy as np
from oawidgets.plantgl import *
from openalea.plantgl.all import Material, Color3, Scene, Viewer
from oawidgets.mtg import *
from alinea.caribu.data_samples import data_path

from openalea.archicrop.simulation import read_sti_file, read_xml_file
from openalea.archicrop.archicrop import ArchiCrop
from openalea.archicrop.display import build_scene, display_scene
from openalea.archicrop.light_it import illuminate
%gui qt

## 1. Retrieve crop model parameters about management

Retrieve crop model parameters regarding the spatial (and temporal) configuration of the crop.

In [2]:
file_tec_xml = 'sorgho_tec.xml'
params_tec = ['densitesem', 'interrang']
tec_stics = read_xml_file(file_tec_xml, params_tec)
sowing_density = tec_stics['densitesem']
inter_row = 0.4

## 2. Retrieve crop model growth and senescence dynamics

From crop-scale data either measured or simulated with a crop model, generate a dictionnary of dictionnaries for each day with the following values :
 - "Thermal time" (float): cumulated thermal time from beginning of simulation to current day (in °C.day)
 - "Phenology" (str): current phenological stage (germination, juvenile, exponential or repro)
 - "Plant leaf area" (float): plant leaf area (in cm²) 
 - "Leaf area increment" (float): leaf area increment compared to previous day (in cm²) 
 - "Plant senescent leaf area" (float): senescent plant leaf area (in cm²) 
 - "Senescent leaf area increment" (float): senescent leaf area increment compared to previous day (in cm²) 
 - "Plant height" (float): plant height (in cm) 
 - "Height increment" (float): height increment compared to previous day (in cm).

In [3]:
stics_output_file = 'mod_ssorghum.sti'
daily_dynamics = read_sti_file(stics_output_file, sowing_density)

# Time series 
# for thermal time, plant leaf area, plant senescent leaf area and plant height
thermal_time = [value["Thermal time"] for value in daily_dynamics.values()]
leaf_area_plant = [value["Plant leaf area"] for value in daily_dynamics.values()]
sen_leaf_area_plant = [value["Plant senescent leaf area"] for value in daily_dynamics.values()]
height_canopy = [value["Plant height"] for value in daily_dynamics.values()]

## 3. Set plant architectural parameters

Set topological, geometrical and developmental parameters, in a range corresponding a given species, found in literature.

In [4]:
nb_phy=10 # number of phytomers on the main stem

# Stem
height=1.5*max(height_canopy) # potential plant height
stem_q=1.1 # parameter for ligule height distribution along axis
diam_base=2.5 # stem base diameter
diam_top=1.5 # stem top diameter

# Leaf blade area 
leaf_area=1.5*max(leaf_area_plant) # potential plant leaf area
rmax=0.9 # parameter for leaf area distribution along axis
skew=0.0005 # parameter for leaf area distribution along axis
wl=0.1 # leaf blade width-to-length ratio 
klig=0.6 # parameter for leaf blade shape
swmax=0.55 # parameter for leaf blade shape
f1=0.64 # parameter for leaf blade shape
f2=0.92 # parameter for leaf blade shape

# Leaf blade position in space
insertion_angle=30 # leaf blade insertion angle
scurv=0.7 # leaf blade relative inflexion point
curvature=90 # leaf blade insertion-to-tip angle
phyllotactic_angle=180 # phyllotactic angle
phyllotactic_deviation=0 # half-deviation to phyllotactic angle

# Development
phyllochron=30 # phyllochron, i.e. stem element appearance rate
plastochron=40 # plastochron, i.e. leaf blade appearance rate
# c'est l'inverse

# Senescence (from crop model parameters)
file_plt_xml = 'proto_sorghum_plt.xml'
params_sen = ['durvieF', 'ratiodurvieI']
sen_stics = read_xml_file(file_plt_xml, params_sen)
lifespan = sen_stics['durvieF'] # leaf lifespan from appearance in exponential phase
lifespan_early = sen_stics['ratiodurvieI'] * lifespan # leaf lifespan from appearance in juvenile phase
leaf_lifespan=[lifespan_early, lifespan]

# Tillering
nb_tillers=3 # number of tillers
tiller_delay=1 # delay, as factor of phyllochron, between the appearance of a phytomer and the appearance of its tiller
tiller_angle=70
reduction_factor=1 # reduction factor between tillers of consecutive order

gravitropism_coefficient = 0.37

## 4. Simulate plant growth for different numbers of tillers

In [5]:
nbs_tillers = [0,1,2,5,10]
plants_tillers = []

for nb_tillers in nbs_tillers:

    plant = ArchiCrop(height=height, 
                      nb_phy=nb_phy,
                      leaf_area=leaf_area,
                      wl=wl, diam_base=diam_base, diam_top=diam_top, 
                      insertion_angle=insertion_angle, scurv=scurv, curvature=curvature, 
                      klig=klig, swmax=swmax, f1=f1, f2=f2, 
                      stem_q=stem_q, rmax=rmax, skew=skew,
                      phyllotactic_angle=phyllotactic_angle,
                      phyllotactic_deviation=phyllotactic_deviation,
                      phyllochron=phyllochron, 
                      plastochron=plastochron, 
                      leaf_lifespan=leaf_lifespan,
                      nb_tillers=nb_tillers, tiller_delay=tiller_delay, reduction_factor=reduction_factor,
                      daily_dynamics=daily_dynamics, 
                      tiller_angle=tiller_angle,
                      gravitropism_coefficient=gravitropism_coefficient)
    
    plant.generate_potential_plant()
    growing_plant = plant.grow_plant()
    plants_tillers.append(list(growing_plant.values()))

## 5. Display MTG of plants with tillers

In [6]:
plot(plants_tillers[0][-1])

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [7]:
plot(plants_tillers[1][-1])

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [8]:
plot(plants_tillers[2][-1])

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [9]:
plot(plants_tillers[3][-1])

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [10]:
plot(plants_tillers[4][-1])

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


## 6. Display MTG of a plant with tillers through time

In [11]:
g = plants_tillers[2][-1]
prop = g.property("start_tt") # inferieur a un temps
plot(g, selection=prop)

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


## 6. Plot 3D scene of plants with tillers

In [12]:
positions = [(0, i*150, 0) for i in range(len(plants_tillers))]
nice_green = Color3((50, 100, 0))
scene, _ = build_scene([plant[-30] for plant in plants_tillers], position=positions, senescence=True, leaf_material = Material(nice_green), stem_material=Material(nice_green))
PlantGL(scene)
Viewer.display(scene)

## 7. Plot 3D scene through time

In [13]:
times = [t for i,t in enumerate(thermal_time) if i%8 == 0]
mean_time = sum(times) / len(times)
positions = [(0, 1*(t-mean_time), 0) for t in times]
nice_green = Color3((50, 100, 0))
scene, _ = build_scene([g for i,g in enumerate(plants_tillers[2]) if i%8 == 0], position=positions, senescence=True, leaf_material = Material(nice_green), stem_material=Material(nice_green))
PlantGL(scene)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

## 8. Illuminate plants with tillers

In [14]:
positions = [(0, i*150, 0) for i in range(len(plants_tillers))]
scene, _ = build_scene([plant[-30] for plant in plants_tillers], position=positions, senescence=True, leaf_material = Material(nice_green), stem_material=Material(nice_green))
cs, ei, eabs, _ = illuminate(scene, light=str(data_path('Turtle46soc.light')))
# scene_ei,values_ei = cs.plot(ei,display=False)
scene_eabs,values_eabs = cs.plot(eabs,display=False)
PlantGL(scene_eabs, group_by_color=False, property=values_eabs)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…

In [15]:
# from IPython.display import Image, display
# import tempfile, os
# from openalea.plantgl.all import Viewer
# fn = tempfile.mktemp(suffix='.png')
# Viewer.saveSnapshot(fn)
# img = Image(fn)
# os.unlink(fn)
# display(img)